In [1]:
import os

def get_next_directory(base_path="output/predict"):
    # Check if the base directory exists
    if not os.path.exists(base_path):
        return base_path
    else:
        # Find the next available numbered directory
        i = 2
        while os.path.exists(f"{base_path}{i}"):
            i += 1
        return f"{base_path}{i}"

In [5]:
NOTEBOOKS_DIR = os.getcwd()

# Create the output directory, if needed
HOME_DIR = os.path.dirname(NOTEBOOKS_DIR)
OUTPUT_DIR = get_next_directory(os.path.join(HOME_DIR, "output/predict"))

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Perform object detection

Perform the inference with the loaded model.

In [6]:
from PIL import Image

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load an official model

source = Image.open(HOME_DIR+"/example-files/IMG_3688.png")

# Predict with the model
results = model(source)  # predict on an image

FileNotFoundError: [Errno 2] No such file or directory: '/Users/andreas/Documents/Projekte/Objekterkennung.yolo11example-files/IMG_3688.png'

# Save cropped images

Saves images for each bounding box.

In [16]:
for result in results:
    result.save_crop(OUTPUT_DIR, "image")